# Start here

In [ ]:
!pip install torch
!pip install textstat
!pip install wordfreq
!pip install random_word
!pip install numpy
!pip install pandas
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torch
import textstat
from wordfreq import word_frequency, random_words
from random_word import Wordnik, RandomWords
import numpy as np
import pandas as pd
import requests
import json
from pprint import pprint
import string
import nltk
from nltk.corpus import wordnet as wn

pd.set_option('max_colwidth', 100)

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Words are looking good now

In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Prodigy/vocab_all.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
# df_entities = pd.read_csv('vocab_entities.csv')
# df_entities.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
def create_quiz_card(df, point, past=[], card=[], n_words=1, pos=''):
    '''
    On rare occasion will return less than n_words if n_words are not or barely available
    given criteria.
    
    `df`: where we select words from
    `point`: starting level of difficulty to supply
    `past`: (optional) list of strings of past synsets already provided
    `card`: (optional) list of strings of synsets on current quiz card
    `n_words`: (optional) how many words to output
    `pos`: (optional) any combination of 'nvsar'
    '''
    RANGE = 2000
    POINT = np.clip(point, RANGE+1, df.shape[0]-RANGE-1)
    outs = [] # list of indices
    df = df.copy().sort_index()
    n_attempts = 0
    
    while len(outs) < n_words:
        n_attempts += 1
        if pos == '':
            samp_idx = df.iloc[POINT-RANGE:POINT+RANGE].sample(1).index[0]
            samp = df.loc[samp_idc]['synset']
        else:
            pos = [*pos]
            tfs = df.iloc[POINT-RANGE:POINT+RANGE]['pos'].isin(pos)
            tfs_idx = tfs.index[tfs]
            samp_idx = df.loc[tfs_idx].sample(1).index[0]
            samp = df.loc[samp_idx]['synset']
            
        if samp in past or samp in card or samp in outs:
            continue
        if samp_idx in outs:
            continue
        if df.loc[samp_idx]['word'] in list(map(lambda x: df.loc[x]['word'], outs)):
            continue
        
        outs.append(samp_idx)
        if n_attempts - n_words > 1:
            RANGE += 200
            POINT = np.clip(POINT, RANGE+1, df.shape[0]-RANGE-1)
        if n_attempts - n_words > 1000:
            break
    return outs

Run below for good words to practice on

In [ ]:
df.loc[create_quiz_card(df, 46000, pos='sa', n_words=5)]

,synset,word,pos,def,examples,lemmas,bad_def
45910,Synset('knowing.s.04'),lettered,s,highly educated; having extensive information or understanding,"['knowing instructors', 'a knowledgeable critic', 'a knowledgeable audience']","['knowing', 'knowledgeable', 'learned', 'lettered', 'well-educated', 'well-read']",False
46197,Synset('materialistic.s.01'),materialistic,s,marked by materialism,[],"['materialistic', 'mercenary', 'worldly-minded']",False
44856,Synset('presumptive.s.02'),presumptive,s,affording reasonable grounds for belief or acceptance,"['presumptive evidence', 'a strong presumptive case is made out']",['presumptive'],False
45818,Synset('circumstantial.s.01'),circumstantial,s,fully detailed and specific about particulars,['a circumstantial report about the debate'],['circumstantial'],False
44562,Synset('repeatable.a.01'),quotable,a,able or fit to be repeated or quoted,"['what he said was not repeatable in polite company', 'he comes up with so many quotable phrases']","['repeatable', 'quotable']",False


# Dataset

In [ ]:
!pip install datasets
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset
from pprint import pprint
from tqdm.notebook import tqdm
import pandas as pd
from sklearn.utils import shuffle

In [ ]:
train = load_dataset('squad', split='train')
valid = load_dataset('squad', split='validation')

example = next(iter(valid))
print('context: ', example['context'], '\n')
print('question: ', example['question'], '\n')
print('answer: ', example['answers']['text'][0])

context:  Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50. 

question:  Which NFL team represented the AFC at Super Bowl 50? 

answer:  Denver Broncos


In [ ]:
long_count = 0
short_count = 0
c,q,a = [], [], []
for i in tqdm(train):
    answer = i['answers']['text'][0]
    if len(answer.split()) > 7:
        long_count += 1
    else:
        short_count += 1
        c.append(i['context'])
        q.append(i['question'])
        a.append(i['answers']['text'][0])
        
df_train = shuffle(pd.DataFrame({'context': c, 'question': q, 'answer': a}))


for i in tqdm(valid):
    answer = i['answers']['text'][0]
    if len(answer.split()) > 7:
        long_count += 1
    else:
        short_count += 1
        c.append(i['context'])
        q.append(i['question'])
        a.append(i['answers']['text'][0])
        
df_valid = shuffle(pd.DataFrame({'context': c, 'question': q, 'answer': a}))

# save
df_train.to_csv('/content/gdrive/My Drive/Prodigy/dataset/squad_train.csv', index=False)
df_valid.to_csv('/content/gdrive/My Drive/Prodigy/dataset/squad_valid.csv', index=False)

  0%|          | 0/87599 [00:00<?, ?it/s]

  0%|          | 0/10570 [00:00<?, ?it/s]

In [ ]:
# load
df_train = pd.read_csv('/content/gdrive/My Drive/Prodigy/dataset/squad_train.csv')
df_valid = pd.read_csv('/content/gdrive/My Drive/Prodigy/dataset/squad_valid.csv')
df_train.head()

,context,question,answer
0,Her first acting role of 2006 was in the comedy film The Pink Panther starring opposite Steve Ma...,"The single, ""Listen"" was featured in which movie?",Dreamgirls
1,Asphalt/bitumen is typically stored and transported at temperatures around 150 °C (302 °F). Some...,To promote what characteristic is diesel oil added to asphalt for shipping?,liquidity
2,"Miami is a major television production center, and the most important city in the U.S. for Spani...","Along with Sony Music Latin, what music recording corporation is located in Miami?",Universal Music Latin Entertainment
3,"The United Nations Organization and its children's agency UNICEF withdrew their staff, saying th...",What is the children's agency of the United Nations Organization?,UNICEF
4,"Somerset has 11,500 listed buildings, 523 scheduled monuments, 192 conservation areas, 41 parks ...",How many monuments on Somerset,523 scheduled monuments


# Tokenizer

In [ ]:
!pip install pytorch_lightning
!pip install transformers
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 798 kB 18.6 MB/s 
     |████████████████████████████████| 529 kB 62.5 MB/s 
     |████████████████████████████████| 125 kB 58.2 MB/s 
     |████████████████████████████████| 87 kB 1.8 MB/s 
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115943 sha256=1dfad84d619c467291b9c1280e66dff0a6261d2b75cab794f54c0cc682ba6ac6
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
Successfully built fire
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 26.4 MB/s 
     |████████████████████████████████| 182 kB 55.8 MB/s 
     |████████████████████████████████| 7.6 MB 45.7 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from transformers import (AdamW, 
                          T5ForConditionalGeneration, 
                          T5Tokenizer, 
                          get_linear_schedule_with_warmup)

import pandas as pd
import numpy as np
from string import punctuation
from pprint import pprint
import copy

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
encoding_example = tokenizer.encode_plus('Hello, welcome Prodigy',
                                        max_length=64,
                                        pad_to_max_length=True,
                                        truncation=True,
                                        return_tensors='pt')
pprint(encoding_example)

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[8774,    6, 2222,  749, 5572,   63,    1,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])}


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


# Prepare dataset

In [ ]:
class QuestionGenerationDataset(Dataset):
    def __init__(self, tokenizer, data, max_len_inp=512, max_len_out=96):
        self.data = data
        self.max_len_inp = max_len_inp
        self.max_len_out = max_len_out
        self.tokenizer = tokenizer
        
        self.inputs = []
        self.targets = []
        self.skipped = 0
        
        self._build()
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, index):
        source_ids = self.inputs[index]['input_ids'].squeeze()
        target_ids = self.targets[index]['input_ids'].squeeze()
        source_mask = self.inputs[index]['attention_mask'].squeeze()
        target_mask = self.targets[index]['attention_mask'].squeeze()
        
        # huggingface requires labels of -100 if label not to be included in loss calculation
        labels = copy.deepcopy(target_ids)
        labels[target_ids==0] = -100
        
        return {'source_ids': source_ids, 'target_ids': target_ids, 'source_mask': source_mask,
               'target_mask': target_mask, 'labels': labels}
    
    def _build(self):
        
        def check_inp_encoding_lengths(row):
            inp = f"<context> {row['context']} <answer> {row['answer']} <end>"
            test_encoding = self.tokenizer.encode_plus(inp, truncation=False, return_tensors='pt')
            if len(test_encoding['input_ids'].squeeze()) > self.max_len_inp:
                return False
            return True
        
        def get_inputs(row):
            inp = f"<context> {row['context']} <answer> {row['answer']} <end>"
            encoding = self.tokenizer.encode_plus(inp, return_tensors='pt', truncation=False,
                                                 max_length=self.max_len_inp, 
                                                 pad_to_max_length=True)
            return encoding

        def get_targets(row):
            tar = f"<question> {row['question']} <end>"
            encoding = self.tokenizer.encode_plus(tar, return_tensors='pt', truncation=False,
                                                 max_length=self.max_len_out, 
                                                 pad_to_max_length=True)
            return encoding
            
        self.checked = self.data.apply(check_inp_encoding_lengths, axis=1)
        self.inputs = self.data.apply(get_inputs, axis=1)[self.checked]
        self.targets = self.data.apply(get_targets, axis=1)[self.checked]

        self.inputs = self.inputs.reset_index(drop=True)
        self.targets = self.targets.reset_index(drop=True)

In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/Prodigy/dataset/squad_train.csv', nrows=1000)
valid_df = pd.read_csv('/content/gdrive/My Drive/Prodigy/dataset/squad_valid.csv', nrows=1000)
train_dataset = QuestionGenerationDataset(tokenizer, train_df)
valid_dataset = QuestionGenerationDataset(tokenizer, valid_df)
# train_dataset[0]

Token indices sequence length is longer than the specified maximum sequence length for this model (678 > 512). Running this sequence through the model will result in indexing errors


# Training

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, model, tokenizer, batch_size):
        super(T5FineTuner, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        
    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None,
               decoder_attention_mask=None, labels=None):
        # don't need decoder_input_ids b/c can be inferred from labels
        outputs = self.model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            decoder_attention_mask = decoder_attention_mask,
            labels = labels
        )
        return outputs
    
    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids = batch['source_ids'], 
            attention_mask = batch['source_mask'], 
            decoder_input_ids = batch['target_ids'],
            decoder_attention_mask = batch['target_mask'], 
            labels = batch['labels']
        )
        loss = outputs[0]
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        print('train_loss: ', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids = batch['source_ids'], 
            attention_mask = batch['source_mask'], 
            decoder_input_ids = batch['target_ids'],
            decoder_attention_mask = batch['target_mask'], 
            labels = batch['labels']
        )
        loss = outputs[0]
        self.log('valid_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.batch_size, num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(valid_dataset, batch_size=self.batch_size, num_workers=4)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer

#### Training this takes too long

So, I'm only training on 1000 data points and doing in small batches.

In [ ]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-38832dda-561b-d505-9bbe-6dad75f80a3f)


In [ ]:
train_df = pd.read_csv('/content/gdrive/My Drive/Prodigy/dataset/squad_train.csv', nrows=5000)
valid_df = pd.read_csv('/content/gdrive/My Drive/Prodigy/dataset/squad_valid.csv', nrows=1000)
train_dataset = QuestionGenerationDataset(tokenizer, train_df)
valid_dataset = QuestionGenerationDataset(tokenizer, valid_df)

t5_model = T5FineTuner(model=model, tokenizer=tokenizer, batch_size=4)
trainer = pl.Trainer(max_epochs=1, accelerator='gpu')
trainer.fit(t5_model)

In [ ]:
t5_model.model.save_pretrained('/content/gdrive/My Drive/Prodigy/model1/model')
tokenizer.save_pretrained('/content/gdrive/My Drive/Prodigy/model1/tokenizer')

('/content/gdrive/My Drive/Prodigy/model1/tokenizer/tokenizer_config.json',
 '/content/gdrive/My Drive/Prodigy/model1/tokenizer/special_tokens_map.json',
 '/content/gdrive/My Drive/Prodigy/model1/tokenizer/spiece.model',
 '/content/gdrive/My Drive/Prodigy/model1/tokenizer/added_tokens.json')

# Testing

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('/content/gdrive/My Drive/Prodigy/model1/model')
tokenizer = T5Tokenizer.from_pretrained('/content/gdrive/My Drive/Prodigy/model1/tokenizer')

In [ ]:
context = '''
WASHINGTON—Congressional leaders agreed Tuesday to head off a nationwide strike by railroad workers, promising to pass legislation quickly that would avert a work stoppage and prevent damage to the economy despite their misgivings about intervening in the dispute. 

Speaker Nancy Pelosi (D., Calif.) said that House lawmakers will aim to pass legislation Wednesday that accepts the original labor union agreement negotiated by Biden administration officials plus additional railway worker benefits added from subsequent negotiations. In the Senate, where bipartisan support would be needed, leaders also said they would aim to quickly pass the legislation once it is sent over from the House, even as some lawmakers pushed for changes to the deal.

“I don’t like going against the ability of unions to strike, but weighing the equities, we must avoid a strike,” said Mrs. Pelosi after meeting with President Biden and congressional leaders of both parties at the White House. “Jobs will be lost, even union jobs will be lost, water will not be safe, product will not be going to market,” she said. 
'''

answer = '''
original labor union agreement
'''
encoded = tokenizer.encode_plus(f'<context> {context} \
                                <answer> {answer} <end>', 
                                truncation=False, return_tensors='pt', max_length=512, pad_to_max_length=True)
encoded

In [ ]:
model.eval()
outs = model.generate(
    input_ids=encoded['input_ids'],
    attention_mask=encoded['attention_mask'],
    max_length=100,
    early_stopping=True,
    num_beams=8,
    num_return_sequences=3
)

[' '.join(tokenizer.decode(out).split()[3:]).strip('<pad></s>end>unk ') for out in outs]

['What was negotiated by Biden administration officials?',
 'What did Biden administration officials negotiate?',
 'What did the House accept?']

# Deploy Model 1 on Gradio

In [ ]:
!pip install --quiet gradio
!pip install --quiet torch
!pip install --quiet pytorch_lightning
!pip install --quiet transformers
!pip install --quiet sentencepiece

import gradio as gr
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from transformers import (AdamW, 
                          T5ForConditionalGeneration, 
                          T5Tokenizer, 
                          get_linear_schedule_with_warmup)

import pandas as pd
import numpy as np
from pprint import pprint
import copy
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('/content/gdrive/My Drive/Prodigy/model1/model/')
tokenizer = T5Tokenizer.from_pretrained('/content/gdrive/My Drive/Prodigy/model1/tokenizer/')

In [ ]:
def generate_questions(context, answer, n_beams=8):
  encoded = tokenizer.encode_plus(f'<context> {context} <answer> {answer} <end>', 
                                  truncation=False, return_tensors='pt', max_length=512, pad_to_max_length=True)
  model.eval()
  outs = model.generate(
    input_ids=encoded['input_ids'],
    attention_mask=encoded['attention_mask'],
    max_length=100,
    early_stopping=True,
    num_beams=n_beams,
    num_return_sequences=5
  )

  return [' '.join(tokenizer.decode(out).split()[3:]).strip('<pad></s>end>unk ') for out in outs]

context_input = gr.Textbox(label='Enter context', value='''WASHINGTON—Congressional leaders agreed Tuesday to head off a nationwide strike by railroad workers, promising to pass legislation quickly that would avert a work stoppage and prevent damage to the economy despite their misgivings about intervening in the dispute. Speaker Nancy Pelosi (D., Calif.) said that House lawmakers will aim to pass legislation Wednesday that accepts the original labor union agreement negotiated by Biden administration officials plus additional railway worker benefits added from subsequent negotiations. In the Senate, where bipartisan support would be needed, leaders also said they would aim to quickly pass the legislation once it is sent over from the House, even as some lawmakers pushed for changes to the deal. “I don’t like going against the ability of unions to strike, but weighing the equities, we must avoid a strike,” said Mrs. Pelosi after meeting with President Biden and congressional leaders of both parties at the White House. “Jobs will be lost, even union jobs will be lost, water will not be safe, product will not be going to market,” she said. ''')
answer_input = gr.Textbox(label='Enter target answer', value='Worker benefits')
# n_questions_input = gr.Radio([1,5,10],label='How many questions to generate?')
n_beams_input = gr.Radio([1,3,5,7,9,11,13], label='How many beams to use?', value=7)
app = gr.Interface(fn=generate_questions, inputs=[context_input, answer_input, n_beams_input], outputs=['text','text','text','text','text'])
app.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# Model 2

#### New dataset, and new purpose

We will use the r/AskReddit subreddit and have question-answer pairs, where we isolate one target word from the answer, provide its definition, and feed into model with teacher-forcing the question generation.

The data for such a specific task is sparse, and this is the best dataset I could find. I'll work on this task later, however, because downloading the subreddit requires ~80GBs in a zipped file. I could not find a better alternative just yet.

In [ ]:
from datasets import load_dataset
import datasets
from torch.utils.data import Dataset, DataLoader
from pprint import pprint
from tqdm import tqdm
import pandas as pd

# Model 3

#### New dataset, and new purpose

Now, the goal is to generate example sentences. The purpose of example sentences is (1) to provide more examples to help understand the word use, and (2) to generate fill-in-the-blank questions. 

For purpose (2), we need to make sure that our quiz card contains words that are similar (i.e., same part of speech so that it's more difficult to differentiate between the words), yet different enough to not be used interchangeably within any given context. For this, we can create a masked word predictive model that gives us predicted prediction probabilities for all the words in our quiz card. If the correct word is of highest probability to be predicted and is significantly above the other words in the quiz card, then this is a valid example sentence.

In [ ]:
!pip install --quiet gradio
!pip install --quiet torch
!pip install --quiet pytorch_lightning
!pip install --quiet transformers
!pip install --quiet datasets
!pip install --quiet sentencepiece

from datasets import load_dataset
import datasets
from torch.utils.data import Dataset, DataLoader
from transformers import (AdamW, 
                          T5ForConditionalGeneration, 
                          T5Tokenizer, 
                          get_linear_schedule_with_warmup,
                          AutoTokenizer,
                          AutoModel)
from pprint import pprint
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from ast import literal_eval
import copy
import pytorch_lightning as pl

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
df = pd.read_csv('/content/gdrive/My Drive/Prodigy/vocab_all.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df['examples'] = df['examples'].apply(literal_eval)

In [ ]:
def create_quiz_card(df, point, past=[], card=[], n_words=1, pos=''):
    '''
    On rare occasion will return less than n_words if n_words are not or barely available
    given criteria.
    
    `df`: where we select words from
    `point`: starting level of difficulty to supply
    `past`: (optional) list of strings of past synsets already provided
    `card`: (optional) list of strings of synsets on current quiz card
    `n_words`: (optional) how many words to output
    `pos`: (optional) any combination of 'nvsar'
    '''
    RANGE = 2000
    POINT = np.clip(point, RANGE+1, df.shape[0]-RANGE-1)
    outs = [] # list of indices
    df = df.copy().sort_index()
    n_attempts = 0
    
    while len(outs) < n_words:
        n_attempts += 1
        if pos == '':
            samp_idx = df.iloc[POINT-RANGE:POINT+RANGE].sample(1).index[0]
            samp = df.loc[samp_idc]['synset']
        else:
            pos = [*pos]
            tfs = df.iloc[POINT-RANGE:POINT+RANGE]['pos'].isin(pos)
            tfs_idx = tfs.index[tfs]
            samp_idx = df.loc[tfs_idx].sample(1).index[0]
            samp = df.loc[samp_idx]['synset']
            
        if samp in past or samp in card or samp in outs:
            continue
        if samp_idx in outs:
            continue
        if df.loc[samp_idx]['word'] in list(map(lambda x: df.loc[x]['word'], outs)):
            continue
        
        outs.append(samp_idx)
        if n_attempts - n_words > 1:
            RANGE += 200
            POINT = np.clip(POINT, RANGE+1, df.shape[0]-RANGE-1)
        if n_attempts - n_words > 1000:
            break
    return outs

In [ ]:
df.loc[create_quiz_card(df, 46000, pos='sa', n_words=5)]

,synset,word,pos,def,examples,lemmas,bad_def
47061,Synset('austere.s.01'),austere,s,severely simple,[a stark interior],"['austere', 'severe', 'stark', 'stern']",False
46474,Synset('dwindling.s.01'),tapering,s,gradually decreasing until little remains,[],"['dwindling', 'tapering', 'tapering_off']",False
47075,Synset('overlying.s.01'),overlying,s,placed on or over something else,[an overlying image],"['overlying', 'superimposed']",False
44201,Synset('beguiled.s.01'),captivated,s,filled with wonder and delight,[],"['beguiled', 'captivated', 'charmed', 'delight...",False
44472,Synset('disruptive.s.01'),tumultuous,s,characterized by unrest or disorder or insubor...,[effects of the struggle will be violent and d...,"['disruptive', 'riotous', 'troubled', 'tumultu...",False


# Dataset

In [ ]:
tokenizer_gpt2 = AutoTokenizer.from_pretrained('gpt2')
tokenizer_gpt2.pad_token = '<pad>'
tokenizer_t5 = AutoTokenizer.from_pretrained('t5-base')

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:166: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
print('\nT5-base\n')
pprint(tokenizer_t5.encode_plus('I like morning jogs', pad_to_max_length=True, truncation=True, max_length=96, return_tensors='pt'))


T5-base

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 'input_ids': tensor([[  27,  114, 1379,    3, 1927,  122,    7,    1,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0, 

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
class ExampleGenerationDatasetT5(Dataset):
  def __init__(self, df, tokenizer, max_len_input=192, max_len_output=96):
    def example_extractor(row):
      word = row['word']
      return [ex for ex in row['examples'] if word in ex]

    def encode_input(row):
      text = f"<word> {row['word']} <pos> {row['pos']} <def> {row['def']} <end>"
      return tokenizer.encode_plus(
          text,
          max_length = max_len_input,
          pad_to_max_length = True,
          truncation = False,
          return_tensors = 'pt'
      )

    def encode_output(row):
      text = f"<example> {row['example'][0]} <end>"
      return tokenizer.encode_plus(
          text,
          max_length = max_len_output,
          pad_to_max_length = True,
          truncation = False,
          return_tensors = 'pt'
      )

    examples = df.apply(example_extractor, axis=1)
    checks = examples.apply(lambda x: len(x) > 0)
    examples = examples[checks].reset_index(drop=True)

    words = df['word'][checks].reset_index(drop=True)
    poss = df['pos'][checks].reset_index(drop=True)
    defs = df['def'][checks].reset_index(drop=True)

    df_new = pd.DataFrame({'word': words, 'pos': poss, 'def': defs, 'example': examples})

    tqdm.pandas()
    self.inputs = df_new.progress_apply(encode_input, axis=1)
    self.outputs = df_new.progress_apply(encode_output, axis=1)



  def __getitem__(self, index):
    src = self.inputs[index]
    trg = self.outputs[index]
    labels = copy.deepcopy(trg['input_ids'])
    labels[labels==0] = -100
    
    return {'source_ids': src['input_ids'].squeeze(),
            'source_mask': src['attention_mask'].squeeze(),
            'target_ids': trg['input_ids'].squeeze(),
            'target_mask': trg['attention_mask'].squeeze(),
            'labels': labels.squeeze()}


  def __len__(self):
    return len(self.inputs)

# Model

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, model, tokenizer, batch_size):
        super(T5FineTuner, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        self.batch_size = batch_size
        
    def forward(self, input_ids, attention_mask=None, decoder_input_ids=None,
               decoder_attention_mask=None, labels=None):
        # don't need decoder_input_ids b/c can be inferred from labels
        outputs = self.model(
            input_ids = input_ids,
            attention_mask = attention_mask,
            decoder_attention_mask = decoder_attention_mask,
            labels = labels
        )
        return outputs
    
    def training_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids = batch['source_ids'], 
            attention_mask = batch['source_mask'], 
            decoder_input_ids = batch['target_ids'],
            decoder_attention_mask = batch['target_mask'], 
            labels = batch['labels']
        )
        loss = outputs[0]
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        print('train_loss: ', loss)
        return loss
    
    def validation_step(self, batch, batch_idx):
        outputs = self.forward(
            input_ids = batch['source_ids'], 
            attention_mask = batch['source_mask'], 
            decoder_input_ids = batch['target_ids'],
            decoder_attention_mask = batch['target_mask'], 
            labels = batch['labels']
        )
        loss = outputs[0]
        self.log('valid_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss
    
    def train_dataloader(self):
        return DataLoader(train_dataset, batch_size=self.batch_size, num_workers=4)
    
    def val_dataloader(self):
        return DataLoader(valid_dataset, batch_size=self.batch_size, num_workers=4)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=3e-4, eps=1e-8)
        return optimizer

In [ ]:
df = df.sample(frac=1).reset_index(drop=True)
df_train = df[10000:]
df_valid = df[:10000]

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

train_dataset = ExampleGenerationDatasetT5(df_train, tokenizer)
valid_dataset = ExampleGenerationDatasetT5(df_valid, tokenizer)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:174: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


  0%|          | 0/21667 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


  0%|          | 0/21667 [00:00<?, ?it/s]

  0%|          | 0/3169 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2310: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


  0%|          | 0/3169 [00:00<?, ?it/s]

In [ ]:
t5_model = T5FineTuner(model=model, tokenizer=tokenizer, batch_size=4)
trainer = pl.Trainer(max_steps=300, accelerator='gpu')
trainer.fit(t5_model)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 222 M 
-----------------------------------------------------
222 M     Trainable params
0     

Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:566: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training: 0it [00:00, ?it/s]

train_loss:  tensor(3.1508, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(2.7210, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(2.1394, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(2.2303, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(2.0022, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(1.9407, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(1.8441, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(2.1594, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(1.4845, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(1.5764, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(1.3825, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(2.0547, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(1.0416, device='cuda:0', grad_fn=<NllLossBackward0>)
train_loss:  tensor(1.3214, device='cuda:0', grad_f

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=300` reached.


In [ ]:
t5_model.model.save_pretrained('/content/gdrive/My Drive/Prodigy/model3/model')
tokenizer.save_pretrained('/content/gdrive/My Drive/Prodigy/model3/tokenizer')

('/content/gdrive/My Drive/Prodigy/model3/tokenizer/tokenizer_config.json',
 '/content/gdrive/My Drive/Prodigy/model3/tokenizer/special_tokens_map.json',
 '/content/gdrive/My Drive/Prodigy/model3/tokenizer/spiece.model',
 '/content/gdrive/My Drive/Prodigy/model3/tokenizer/added_tokens.json')

# Testing

In [ ]:
model = T5ForConditionalGeneration.from_pretrained('/content/gdrive/My Drive/Prodigy/model3/model')
tokenizer = T5Tokenizer.from_pretrained('/content/gdrive/My Drive/Prodigy/model3/tokenizer')

In [ ]:
text = '<word> adjacent <pos> n <def> by its side <end>'
encoded = tokenizer.encode_plus(
            text,
            max_length = 192,
            pad_to_max_length = True,
            truncation = True,
            return_tensors = 'pt'
          )
encoded

{'input_ids': tensor([[    3,     2,  6051,  3155, 12487,     3,     2,  2748,  3155,     3,
            29,     3,     2,   221,    89,  3155,    57,   165,   596,     3,
             2,   989,  3155,     1,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
model.eval()
outs = model.generate(
    input_ids=encoded['input_ids'],
    attention_mask=encoded['attention_mask'],
    max_length=96,
    early_stopping=True,
    num_beams=11,
    num_return_sequences=3
)

[tokenizer.decode(out)[tokenizer.decode(out).index('example>')+8:tokenizer.decode(out).index('<unk>', tokenizer.decode(out).index('example>')+8)].strip() \
 for out in outs]

['adjacent buildings', 'adjacent trees', 'adjacent land']

# Deployment

In [ ]:
!pip install --quiet gradio
!pip install --quiet torch
!pip install --quiet pytorch_lightning
!pip install --quiet transformers
!pip install --quiet datasets
!pip install --quiet sentencepiece

from datasets import load_dataset
import datasets
from torch.utils.data import Dataset, DataLoader
from transformers import (AdamW, 
                          T5ForConditionalGeneration, 
                          T5Tokenizer, 
                          get_linear_schedule_with_warmup,
                          AutoModel)
from pprint import pprint
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from ast import literal_eval
import copy
import pytorch_lightning as pl
import gradio as gr

from google.colab import drive
drive.mount('/content/gdrive')

     |████████████████████████████████| 451 kB 37.0 MB/s 
     |████████████████████████████████| 132 kB 78.3 MB/s 
     |████████████████████████████████| 212 kB 79.0 MB/s 
     |████████████████████████████████| 127 kB 77.9 MB/s 
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
model = T5ForConditionalGeneration.from_pretrained('/content/gdrive/My Drive/Prodigy/model3/model')
tokenizer = T5Tokenizer.from_pretrained('/content/gdrive/My Drive/Prodigy/model3/tokenizer')

In [ ]:
def generate_questions(word, pos, def_, n_beams):
  encoded = tokenizer.encode_plus(f'<word> {word} <answer> {pos} <def> {def_} <end>', 
                                  truncation=False, return_tensors='pt', max_length=192, pad_to_max_length=True)
  model.eval()
  outs = model.generate(
    input_ids=encoded['input_ids'],
    attention_mask=encoded['attention_mask'],
    max_length=96,
    early_stopping=True,
    num_beams=n_beams,
    num_return_sequences=3
  )

  return [tokenizer.decode(out)[tokenizer.decode(out).index('example>')+8:\
                                tokenizer.decode(out).index('<unk>', tokenizer.decode(out).index('example>')+8)].strip() \
          for out in outs]

word_input = gr.Textbox(label='Enter word', value='rococo')
def_input = gr.Textbox(label='Enter definition', value='having excessive asymmetrical ornamentation')
pos_input = gr.Radio(['n','v','s','a','r'], label="What is this word's part of speech?", value='s')


n_beams_input = gr.Radio([1,3,5,7,9,11,13], label='How many beams to use?', value=7)
app = gr.Interface(fn=generate_questions, inputs=[word_input, def_input, pos_input, n_beams_input], outputs=['text','text','text','text','text'])
app.launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>